In [ ]:
import pandas as pd
import collections

In [ ]:
#https://dados.educacao.sp.gov.br/search/field_topic/desafios-seduc-164

# FATO
- Avaliacao do Saresp
- Ano
- Valor (medprof)
- Dim disciplina (serie/periodo/conteudo) - TODO: incluir o professor aqui.
- Dim equipe de gestores - tabela com Ids e uma auxiliar de detalhamentos
- Dim escola

# FUNCOES

In [273]:
def limpa_serie_ano(serie_ano):
    if serie_ano == '9Âº Ano EF' or serie_ano == '9º Ano EF':
        return 'EF-9'
    elif serie_ano == '7Âº Ano EF':
        return 'EF-7'
    elif serie_ano == 'EM-3Âª sÃ©rie' or serie_ano == 'EM-3ª série':
        return 'EM-3'
    elif serie_ano == '3Âº Ano EF' or serie_ano == '3º Ano EF':
        return 'EF-3'
    elif serie_ano == '5Âº Ano EF' or serie_ano == '5º Ano EF':
        return 'EF-5'
    
def limpa_ds_comp(ds_comp):
    if ds_comp == 'LÃ\x8dNGUA PORTUGUESA' or ds_comp == 'LÍNGUA PORTUGUESA':
        return 'PORTUGUES'
    elif ds_comp == 'MATEMÃ\x81TICA' or ds_comp == 'MATEMÁTICA':
        return 'MATEMATICA'
    elif ds_comp == 'CIÃ\x8aNCIAS':
        return 'CIENCIAS'    
    
def enulo(valor):
    return str(valor) == 'nan'   

# CARREGAMENTO DOS DADOS

In [ ]:
# SARESP
arquivo = 'dados/saresp/SARESP_escolas_2014.csv'
df_2014 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2014['ano'] = 2014
arquivo = 'dados/saresp/SARESP_escolas_2015.csv'
df_2015 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2015['ano'] = 2015
arquivo = 'dados/saresp/SARESP_escolas_2016.csv'
df_2016 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2016['ano'] = 2016
arquivo = 'dados/saresp/SARESP_escolas_2017.csv'
df_2017 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2017['ano'] = 2017
arquivo = 'dados/saresp/SARESP_escolas_2018.csv'
df_2018 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2018['ano'] = 2018
saresp = df_2014.append(df_2015).append(df_2016).append(df_2017).append(df_2018)
saresp = saresp[saresp.medprof.notnull()] # nao estou interessado se nao tiver o valor do saresp

In [ ]:
# GESTORES
arquivo = 'dados\lideres\lideres_DIRETORES DE ESCOLA.csv'
diretor = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_VICE_DIRETOR.csv'
vice = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_PROFESSOR_COORDENADOR.csv'
coordenador = pd.read_csv(arquivo, sep=';')
vice = vice.drop('Unnamed: 16',axis=1)

In [ ]:
# IDESP
df_idesp_anosiniciais = pd.read_csv('dados\idesp/IDESP_Escolas_2007_2018_AI.CSV',sep=',')
df_idesp_anosfinais = pd.read_csv('dados\idesp/IDESP_Escolas_2007_2018_AF.CSV',sep=',')
df_idesp_ensinomedio = pd.read_csv('dados\idesp/IDESP_Escolas_2007_2018_EM.CSV',sep=',')

In [ ]:
df_idesp = df_idesp_anosiniciais.append(df_idesp_anosfinais).append(df_idesp_ensinomedio)

# GERANDO DIMENSAO IDESP

In [265]:
df_idesp.columns

Index(['CODIGO CIE', 'CODIGO INEP', 'DIRETORIA', 'ESCOLA', 'MUNICIPIO',
       'NIVEL ENSINO', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [278]:
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
LISTA_ANO_ANTERIOR = ['2013', '2014', '2015', '2016', '2017']
idesp = {}
idesp['id_gestao'] = []
idesp['id_escola'] = []
idesp['ano_gestao'] = []
idesp['nivel_ensino'] = []
idesp['nota_ano'] = []
idesp['nota_anterior'] = []

for _, row in df_idesp.iterrows():
    for ano,ano_anterior in zip(LISTA_ANOS, LISTA_ANO_ANTERIOR):
        if enulo(row.loc[ano]):
            continue
        idesp_anterior = row.loc[ano_anterior]
        if enulo(idesp_anterior):
            idesp_anterior = row.loc[ano]
            
        idesp['id_gestao'].append('esc-' + str(row.loc['CODIGO CIE']) + '-ano-' + str(ano))
        idesp['id_escola'].append(row.loc['CODIGO CIE'])
        idesp['ano_gestao'].append(ano)
        idesp['nivel_ensino'].append(row.loc['NIVEL ENSINO'])
        idesp['nota_ano'].append(row.loc[ano])
        idesp['nota_anterior'].append(idesp_anterior)

idesp = pd.DataFrame(data=idesp)

In [314]:
idesp_group = {}
idesp_group['id_gestao'] = []
idesp_group['id_escola'] = []
idesp_group['ano_gestao'] = []
idesp_group['nota_ano'] = []
idesp_group['variacao'] = []

for nome, dados in idesp.groupby('id_gestao'):
    nota_ano = dados.nota_ano.mean()
    nota_anterior = dados.nota_anterior.mean()
    variacao = int(100 * (nota_ano - nota_anterior) / nota_anterior)
    idesp_group['id_gestao'].append(nome)
    idesp_group['id_escola'].append(dados.id_escola.iloc[0])
    idesp_group['ano_gestao'].append(int(dados.ano_gestao.iloc[0]))
    idesp_group['nota_ano'].append(nota_ano)
    idesp_group['variacao'].append(variacao)
    
idesp_group = pd.DataFrame(data=idesp_group)

In [312]:
idesp_group.variacao.describe()

count    25111.000000
mean         7.362311
std         19.715685
min        -70.000000
25%         -4.000000
50%          4.000000
75%         16.000000
max        379.000000
Name: variacao, dtype: float64

### ESTUDO DAS VARIACOES

In [320]:
idesp_group[idesp_group.variacao > 200]

,id_gestao,id_escola,ano_gestao,nota_ano,variacao
1033,esc-12488-ano-2015,12488,2015,2.87,304
4102,esc-2008-ano-2016,2008,2016,2.46,261
7679,esc-29269-ano-2015,29269,2015,7.96,237
11831,esc-40551-ano-2016,40551,2016,2.25,324
12296,esc-4157-ano-2014,4157,2014,3.50,379
12667,esc-42500-ano-2018,42500,2018,3.70,345
12717,esc-42626-ano-2018,42626,2018,3.08,201


In [317]:
df_idesp[df_idesp.loc[:,'CODIGO CIE'] == 40551]

,CODIGO CIE,CODIGO INEP,DIRETORIA,ESCOLA,MUNICIPIO,NIVEL ENSINO,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2253,40551,35040551.0,ITAPEVI,LENIO VIEIRA DE MORAES PROFESSOR ...,BARUERI,ENSINO MEDIO,0.76,1.14,0.96,0.86,1.3,1.15,1.27,1.0,0.53,2.25,1.77,1.56


# GERANDO DIMENSAO SARESP

In [ ]:
saresp.SERIE_ANO = saresp.SERIE_ANO.apply(limpa_serie_ano)
saresp.ds_comp = saresp.ds_comp.apply(limpa_ds_comp)
saresp.medprof = saresp.medprof \
                .apply(lambda x: str(x).replace('.','').replace(',','.')) \
                .astype(float) 

In [ ]:
df_saresp = {}
df_saresp['id_gestao'] = []
df_saresp['id_escola'] = []
df_saresp['ano_gestao'] = []
df_saresp['serie_ano'] = []
df_saresp['periodo'] = []
df_saresp['disciplina'] = []
df_saresp['medprofici'] = []

for _, row in saresp.iterrows():
    df_saresp['id_gestao'].append('esc-' + str(row.CODESC) + '-ano-' + str(row.ano))
    df_saresp['id_escola'].append(row.CODESC)
    df_saresp['ano_gestao'].append(row.ano)
    df_saresp['serie_ano'].append(row.SERIE_ANO)
    df_saresp['periodo'].append(row.periodo)
    df_saresp['disciplina'].append(row.ds_comp)
    df_saresp['medprofici'].append(row.medprof)

df_saresp = pd.DataFrame(data=df_saresp) 

In [ ]:
df_saresp.head(3)

In [ ]:
df_saresp

In [ ]:
saresp_medias = {}
saresp_medias['id_gestao'] = []
saresp_medias['media'] = []
saresp_medias['id_escola'] = []
for nome, dados in df_saresp.groupby('id_gestao'):
    saresp_medias['id_gestao'].append(nome)
    saresp_medias['media'].append(dados.medprofici.mean())
    saresp_medias['id_escola'].append(dados.id_escola.iloc[0])

saresp_medias = pd.DataFrame(data = saresp_medias)

In [ ]:
saresp_medias = pd.DataFrame(data = saresp_medias)

In [ ]:
saresp_medias.head(10)

In [ ]:
dados

# GERANDO DIMENSAO GESTORES
- cd_escola
- ano
- diretoria
- id_diretor - detalhamento aparece em outro lugar
- id_vice
- id_coordenador

### IMPORTANTE: A gestao 'esc-18314-ano-2018' teve 8 diretores, meio complicado de avaliar, ne.
- equipe completa? 
- num_mudancas de gestores
- atribuir numeros da proeficiencia como 1,2,3,4,5 na ordem de qualidade dos anos analisados.
- AVALIAR TEMPO COM A MESMA EQUIPE DE GESTORES

In [ ]:
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
def foi_gestor(ano):
    return ano == 'SIM'

In [ ]:
df_gestao = {}
df_gestao['id_gestao'] = []
df_gestao['id_escola'] = []
df_gestao['id_gestor'] = []
df_gestao['ano_gestao'] = []
df_gestao['diretoria'] = []
df_gestao['funcao'] = []

for _, row in diretor.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('DIRETOR')

for _, row in vice.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('VICE')
            
for _, row in coordenador.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('COORDENADOR')
            
            

In [ ]:
df_gestao = pd.DataFrame(df_gestao)

In [ ]:
k = 0
contagem_gestores = collections.Counter(df_gestao.id_gestao)
for id_gestao in contagem_gestores:
    if contagem_gestores[id_gestao] == 3:
        k += 1
        #print(id_gestao)
        #break

In [ ]:
df_gestao[df_gestao.id_gestao == 'esc-472505-ano-2014']

In [ ]:
len(set(df_gestao.id_gestao))

In [ ]:
df_diretor = pd.DataFrame(df_diretor)

In [ ]:
df_diretor.head(3)

In [ ]:
len(df_diretor)

In [ ]:
collections.Counter(df_diretor.id_gestao).most_common(3)

In [ ]:
df_diretor[df_diretor.id_gestao == 'esc-7985-ano-2018']

# TRABALHANDO

In [ ]:
gest = diretor.append(vice).append(coordenador)

In [ ]:
collections.Counter(gest.loc[:,'2014'])

In [ ]:
coordenador.head(3)

In [ ]:
saresp.head(3)

In [ ]:
saresp.medprof = saresp.medprof \
                .apply(lambda x: str(x).replace('.','').replace(',','.')) \
                .astype(float)

In [ ]:
saresp.medprof.describe()

In [ ]:
saresp[saresp.CODESC == 924933].head(3)

In [ ]:
len(set(gest.CD_ESCOLA).intersection(set(saresp.CODESC)))

In [ ]:
collections.Counter(saresp.SERIE_ANO.apply(limpa_serie_ano))

In [ ]:
df_saresp.head(5)

In [ ]:
set(collections.Counter(df_saresp.NomeDepBol).keys())

In [ ]:
arquivo = 'dados\lideres\lideres_DIRETORES DE ESCOLA.csv'
gest = pd.read_csv(arquivo, sep=';') # PELO CD_ESCOLA EU ACHO - PELO ID_INTERNO EU VOU NO SERVIDOR

In [ ]:
arquivo = 'dados/saresp/SARESP_escolas_2014.csv'
df_saresp = pd.read_csv(arquivo, sep=';', encoding='latin-1') # CODESC

In [ ]:
gest.head(3)